# Taller Text Mining

In [1]:
import sys, os, csv, glob, json, uuid, pickle, math
import nltk
import numpy as np, scipy, pandas as pd
from operator import itemgetter
from IPython.display import HTML, display
import tabulate
from tqdm import tqdm
from pyspark.ml.feature import Word2Vec, Word2VecModel


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/njaram15/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/njaram15/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
URI = sc._gateway.jvm.java.net.URI
Path = sc._gateway.jvm.org.apache.hadoop.fs.Path
FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
fs = FileSystem.get(URI("hdfs://hdpjupyter.dis.eafit.edu.co:8020/user/njaram15/"), sc._jsc.hadoopConfiguration())

# We can now use the Hadoop FileSystem API (https://hadoop.apache.org/docs/current/api/org/apache/hadoop/fs/FileSystem.html)
#fs.listStatus(Path('/user/njaram15/'))


In [4]:
CONTENT_INDEX = 9

userPath = '/user/njaram15/'

csv.field_size_limit(sys.maxsize)
CONTENT_PATH = 'inputs/contents/'
TOKENS_PATH = 'inputs/tokens/'
CENTROIDS_PATH = 'inputs/centroids/'
MODEL_PATH = 'model/'

fs.mkdirs(Path(CONTENT_PATH))
fs.mkdirs(Path(TOKENS_PATH))
fs.mkdirs(Path(CENTROIDS_PATH))
fs.mkdirs(Path(MODEL_PATH))

True

In [26]:
df = spark.read.csv(userPath+'/inputs/*.csv', header=True)

In [27]:
df.show()

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|_c0|   id|               title|   publication|              author|      date|  year|month| url|             content|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|  0|17283|House Republicans...|New York Times|          Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
|  1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0|  6.0|null|After the bullet ...|
|  2|17285|Tyrus Wong, ‘Bamb...|New York Times|        Margalit Fox|2017-01-06|2017.0|  1.0|null|When Walt Disney’...|
|  3|17286|Among Deaths in 2...|New York Times|    William McDonald|2017-04-10|2017.0|  4.0|null|Death may be the ...|
|  4|17287|Kim Jong-un Says ...|New York Times|       Choe Sang-Hun|2017-01-02|2017.0|  1.0|null|SEOUL, South Kore...|
|  5|17288|Sick With a Cold,...|New York Times| 

In [46]:
count = 0

for line in df.rdd.collect():
    count = count + 1
    content = line['content']
    cname = CONTENT_PATH + str(count) + '.txt'
    tname = TOKENS_PATH + str(count) + '.tokens'
    #write_s3file(cname,content)
    if not fs.exists(Path(userPath+cname)):
        sc.parallelize([content]).saveAsTextFile(cname)    
    sentences = ""
    for sentence in nltk.sent_tokenize(content):
        sentences = sentences + sentence.lower() + "\n"

    #write_s3file(tname, sentences)
    if not fs.exists(Path(userPath+tname)):
        sc.parallelize([sentences]).saveAsTextFile(tname)    

WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been d

In [29]:
from nltk.stem.porter import *

def cleanStopWordAndStemming(sentences):
    stop_words = stopwords.words('english')
    stemmer = PorterStemmer()
    sen = []
    for sentence in sentences:
        senten = [word for word in sentence if word not in stop_words]
        singles = [stemmer.stem(token) for token in senten]
        sen.append(singles)
    return sen

In [30]:
sentences = []
df2 = spark.read.text(userPath+'/inputs/tokens/*.tokens')
for line in df2.collect():
    sentences.append(nltk.word_tokenize(line['value'].strip()))

In [32]:
sentences = cleanStopWordAndStemming(sentences)
print(sentences[0])

['joachim', 'neander', 'calvinist', 'theologian', 'often', 'hike', 'valley', 'outsid', 'düsseldorf', ',', 'germani', ',', 'write', 'hymn', '.']


In [33]:
inputs = [(x,) for x in sentences]
doc = spark.createDataFrame(inputs, ["sentence"])
word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="sentence", outputCol="model")
model = word2Vec.fit(doc)
model.getVectors().show()

+-----------+--------------------+
|       word|              vector|
+-----------+--------------------+
|    rahmani|[-0.0513312593102...|
|      brink|[0.15214513242244...|
|   promenad|[0.16506512463092...|
|    acronym|[0.06156763806939...|
|  forgotten|[0.27298960089683...|
|     justif|[-0.0942504927515...|
|     teresa|[0.04786748439073...|
|      lover|[0.08759211003780...|
|     comedi|[0.03018949925899...|
|  regularli|[0.25700578093528...|
|      fanci|[0.01558598503470...|
|       elit|[0.34433779120445...|
|    speaker|[0.11496806144714...|
|       chee|[0.13994784653186...|
|       lion|[0.06953834742307...|
|       rate|[0.30278375744819...|
|     pepper|[0.08770198374986...|
|       2014|[0.00547009333968...|
|uncertainti|[0.31668323278427...|
|nonetheless|[0.14262537658214...|
+-----------+--------------------+
only showing top 20 rows



In [34]:
if not fs.exists(Path(userPath+'/model/word2vec-model')):
    model.save(userPath+'/model/word2vec-model')

In [35]:
model = None
model = Word2VecModel.load(userPath+'/model/word2vec-model')

In [36]:
words = model.getVectors().select('word').collect()
vocab = dict([(v.word, k) for k, v in enumerate(words)])
#write_s3file('taller2/model/w2v-lc-vocab.json', json.dumps(vocab))
if not fs.exists(Path(userPath+'/model/w2v-lc-vocab.json')):
    sc.parallelize([json.dumps(vocab)]).saveAsTextFile(userPath+'/model/w2v-lc-vocab.json')

In [37]:
from pyspark.sql.functions import format_number as fmt

vectors = model.getVectors()
t = vectors.filter(vectors.word == 'sport').select('vector').collect()[0]['vector']
# como senate y alabama no están en el dataset de prueba se simulan con ceros
s = np.array([0.0,0.0,0.0,0.0,0.0])
a = np.array([0.0,0.0,0.0,0.0,0.0])
r = t+s-a
# r = a-t+s
model.findSynonyms(r, 10).select("word", fmt("similarity", 5).alias("similarity")).show()

+---------+----------+
|     word|similarity|
+---------+----------+
|    sport|   1.00000|
|    chunk|   0.99398|
|       57|   0.99322|
|  worship|   0.99149|
|   flight|   0.98690|
|intercept|   0.98636|
|    bride|   0.98564|
|    exten|   0.98529|
|  toddler|   0.98524|
|    doubl|   0.98068|
+---------+----------+



In [47]:

from  pyspark.sql.functions import input_file_name
df3 = spark.read.text(userPath+'/inputs/contents/*.txt')
df3 = df3.withColumn("filename", input_file_name())
df3.show()

+--------------------+--------------------+
|               value|            filename|
+--------------------+--------------------+
|Joachim Neander w...|hdfs://hdpjupyter...|
|WASHINGTON  —   W...|hdfs://hdpjupyter...|
|When Indira Islas...|hdfs://hdpjupyter...|
|On the morning of...|hdfs://hdpjupyter...|
|In the fall of 20...|hdfs://hdpjupyter...|
|Hours before the ...|hdfs://hdpjupyter...|
|President Obama d...|hdfs://hdpjupyter...|
|One night six yea...|hdfs://hdpjupyter...|
|After the bullet ...|hdfs://hdpjupyter...|
|The Season 7 “Rea...|hdfs://hdpjupyter...|
|On the night of N...|hdfs://hdpjupyter...|
|WASHINGTON  —   O...|hdfs://hdpjupyter...|
|Canada, our No. 1...|hdfs://hdpjupyter...|
|A   of   lead exp...|hdfs://hdpjupyter...|
|Updated: 11:50 p....|hdfs://hdpjupyter...|
|• Hundreds of tho...|hdfs://hdpjupyter...|
|Thousands of year...|hdfs://hdpjupyter...|
|GLEN ELDER, Kan. ...|hdfs://hdpjupyter...|
|Danny Cahill stoo...|hdfs://hdpjupyter...|
|They called him t...|hdfs://hdp

In [51]:
import time

t0 = time.time()
dic = {k:v for k,v in vectors.collect()}
for fname in df3.rdd.collect():
    rdd = sc.parallelize([fname['value']])
    centroid_in = list(rdd.map(lambda x: [dic[w] if w in dic.keys() else [0.0,0.0,0.0,0.0,0.0] for w in x.lower().split(' ')]).map(lambda x: np.mean(x, axis=0)).collect()[0])
    out_dict = { fname['filename'] : centroid_in}
    json_file = '/inputs/centroids/' + fname['filename'].split('/')[-2].replace('.txt', '.json')
    if not fs.exists(Path(userPath+json_file)):
        sc.parallelize([json.dumps(out_dict)]).saveAsTextFile(userPath+json_file)

print('time elapsed: ', time.time()-t0)

time elapsed:  1682.779007434845


In [53]:
sents = [(s,) for s in sentences]
df4 = spark.createDataFrame(sents)
dictionary = df4.rdd.map(lambda x: x[0]).reduce(lambda x,y: list(np.unique(list(x)+list(y))))

In [82]:
centroid_dict = {}
dfCentroide = spark.read.text(userPath+'inputs/centroids/*.json')
dfCentroide.show()
for fname in dfCentroide.rdd.collect():
    d = json.loads(fname['value'])
    centroid_dict.update(d)

+--------------------+
|               value|
+--------------------+
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
|{"hdfs://hdpjupyt...|
+--------------------+
only showing top 20 rows



In [83]:
clean_centroid_dict = {k: centroid_dict[k] for k in centroid_dict if not np.isnan(centroid_dict[k][0]).any()}

In [57]:
def get_embedding(x, out=False):
    if x in dictionary:
        return vectors.filter(vectors.word == x).select('vector').collect()[0]['vector']
    else:
        return np.zeros(100)

In [58]:
def score_document(q_embeddings, d_centroid):
    individual_csims = [(1 - scipy.spatial.distance.cosine(qin, d_centroid)) for qin in q_embeddings]
    return (sum(individual_csims)/len(q_embeddings))

In [85]:
documents = []
df5 = spark.read.text(userPath+'/inputs/contents/*.txt')
for fname in df5.rdd.collect():
    documents.append(nltk.word_tokenize(fname['value'].strip()))

In [86]:
query = 'mother buys'
query_words = nltk.word_tokenize(query.lower())
query_ins = [get_embedding(x) for x in query_words]
q_len = len(query_ins)
print('Num words in query: ', len(query_words), 'Num query word in vectors: ', q_len)

Num words in query:  2 Num query word in vectors:  2


In [87]:
scores_in_in = []
for k,v in clean_centroid_dict.items():
    scores_in_in.append((k, score_document(query_ins, v[0])))

scores_in_in = sorted(scores_in_in, key=itemgetter(1), reverse=True)

/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [88]:
print('TOP 5 IN-IN:')
top_5_in_in = [x[0] for x in scores_in_in[:5]]

for fname in top_5_in_in:
    print(fname)

TOP 5 IN-IN:
hdfs://hdpjupyter.dis.eafit.edu.co:8020/user/njaram15/inputs/contents/324.txt/part-00001
hdfs://hdpjupyter.dis.eafit.edu.co:8020/user/njaram15/inputs/contents/116.txt/part-00001
hdfs://hdpjupyter.dis.eafit.edu.co:8020/user/njaram15/inputs/contents/280.txt/part-00001
hdfs://hdpjupyter.dis.eafit.edu.co:8020/user/njaram15/inputs/contents/283.txt/part-00001
hdfs://hdpjupyter.dis.eafit.edu.co:8020/user/njaram15/inputs/contents/452.txt/part-00001
